In [3]:
!pip install xai-sdk --quiet
!pip install transformers torch sentencepiece langdetect PyPDF2 scikit-learn gradio fastapi uvicorn nest-asyncio arabic-reshaper python-bidi pyngrok arabert==1.0.1 nltk google-generativeai python-docx spacy textblob --quiet

import nest_asyncio
import os
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from transformers import pipeline
import langdetect
from arabert.preprocess import ArabertPreprocessor
import gradio as gr
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
import uvicorn
import threading
import asyncio
import requests
from pyngrok import ngrok
from google.colab import userdata
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from google.generativeai import types
import PyPDF2
from docx import Document
import tempfile
import json
from datetime import datetime
from collections import Counter, defaultdict

# Configuration initiale
print("🔧 Initialisation du système TuniSpeak avec NLP et ML...")

# Fetch GEMINI_API_KEY
GEMINI_API_KEY = None
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        print("⚠️ Warning: GEMINI_API_KEY secret not found. Generative AI fallback will not work.")
except Exception as e:
    print(f"❌ Error fetching GEMINI_API_KEY secret: {e}")

# Download NLTK resources
try:
    nltk.download('punkt_tab')
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    print("✅ NLTK resources downloaded")
except Exception as e:
    print(f"⚠️ Erreur lors du téléchargement des ressources NLTK : {e}")

# Patch pour exécuter uvicorn dans Colab
nest_asyncio.apply()

# =============================================================================
# MODULE D'APPRENTISSAGE - Collecte et analyse des interactions
# =============================================================================

class LearningModule:
    """Module d'apprentissage qui améliore le système au fil du temps"""

    def __init__(self):
        self.interactions_log = []
        self.feedback_log = []
        self.intent_classifier = None
        self.user_patterns = defaultdict(list)
        self.learning_data_file = "/content/learning_data.json"
        self.load_learning_data()

    def _get_language_name(self, code):
        """Convertit le code langue en nom complet"""
        language_map = {
            'fr': 'Français',
            'ar': 'Arabe',
            'dar': 'Darija Tunisienne'
        }
        return language_map.get(code, code)

    def load_learning_data(self):
        """Charge les données d'apprentissage précédentes"""
        try:
            if os.path.exists(self.learning_data_file):
                with open(self.learning_data_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    self.interactions_log = data.get('interactions', [])
                    self.feedback_log = data.get('feedback', [])
                print(f"✅ Données d'apprentissage chargées: {len(self.interactions_log)} interactions")
        except Exception as e:
            print(f"⚠️ Impossible de charger les données d'apprentissage: {e}")

    def save_learning_data(self):
        """Sauvegarde les données d'apprentissage"""
        try:
            data = {
                'interactions': self.interactions_log[-1000:],  # Garder les 1000 dernières
                'feedback': self.feedback_log[-500:]  # Garder les 500 derniers
            }
            with open(self.learning_data_file, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"⚠️ Erreur sauvegarde apprentissage: {e}")

    def log_interaction(self, question, answer, source, language, similarity_score=None):
        """Enregistre une interaction pour l'apprentissage"""
        interaction = {
            'timestamp': datetime.now().isoformat(),
            'question': question,
            'answer': answer[:200],  # Tronquer pour économiser l'espace
            'source': source,
            'language': language,
            'similarity_score': similarity_score
        }
        self.interactions_log.append(interaction)

        # Sauvegarder tous les 10 interactions
        if len(self.interactions_log) % 10 == 0:
            self.save_learning_data()

    def log_feedback(self, question, was_helpful, comment=None):
        """Enregistre le feedback utilisateur"""
        feedback = {
            'timestamp': datetime.now().isoformat(),
            'question': question,
            'was_helpful': was_helpful,
            'comment': comment
        }
        self.feedback_log.append(feedback)
        self.save_learning_data()

    def analyze_patterns(self):
        """Analyse les patterns d'utilisation"""
        if len(self.interactions_log) == 0:
            return "📊 **Rapport d'Apprentissage**\n━━━━━━━━━━━━━━━━━━━━━━━━━━\nAucune donnée disponible. Posez quelques questions pour générer un rapport !"

        # Analyse toujours, même avec peu de données
        languages = [i['language'] for i in self.interactions_log]
        lang_counts = Counter(languages)

        sources = [i['source'] for i in self.interactions_log]
        source_counts = Counter(sources)

        # Calculer les scores de similarité (uniquement pour FAQ)
        faq_scores = [i['similarity_score'] for i in self.interactions_log
                      if i['source'] == 'FAQ' and i['similarity_score'] is not None]
        avg_faq_score = np.mean(faq_scores) if faq_scores else 0

        # Calculer les statistiques de feedback
        helpful_feedback = [f for f in self.feedback_log if f['was_helpful']]
        helpful_rate = len(helpful_feedback) / len(self.feedback_log) * 100 if self.feedback_log else 0

        # Créer le rapport
        report_lines = [
            "📊 **Rapport d'Apprentissage**",
            "━━━━━━━━━━━━━━━━━━━━━━━━━━",
            f"📈 Total interactions: {len(self.interactions_log)}",
            f"💬 Feedbacks reçus: {len(self.feedback_log)}",
            f"👍 Taux de satisfaction: {helpful_rate:.1f}%",
            "",
            "🌍 **Distribution des langues:**"
        ]

        # Ajouter les langues
        for lang, count in lang_counts.most_common():
            percentage = count / len(languages) * 100
            report_lines.append(f"  • {self._get_language_name(lang)}: {count} ({percentage:.1f}%)")

        report_lines.append("")
        report_lines.append("📚 **Sources utilisées:**")

       # Ajouter les sources
        for source, count in source_counts.most_common():
            percentage = count / len(sources) * 100
            report_lines.append(f"  • {source}: {count} ({percentage:.1f}%)")

       # Ajouter les performances si pertinent
        if faq_scores:
           report_lines.append("")
           report_lines.append("🎯 **Performance FAQ:**")
           report_lines.append(f"  • Score de similarité moyen: {avg_faq_score:.3f}")
           report_lines.append(f"  • Nombre de réponses FAQ: {len(faq_scores)}")

    # Ajouter des conseils si peu de données
        if len(self.interactions_log) < 10:
          report_lines.append("")
          report_lines.append("💡 **Conseil:** Continuez à poser des questions pour obtenir une analyse plus précise!")

        return "\n".join(report_lines)

    def train_intent_classifier(self, questions, intents):
        """Entraîne un classificateur d'intentions"""
        try:
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.naive_bayes import MultinomialNB
            from sklearn.pipeline import Pipeline

            # Créer un pipeline
            self.intent_classifier = Pipeline([
                ('tfidf', TfidfVectorizer(max_features=1000)),
                ('clf', MultinomialNB())
            ])

            # Entraîner
            X_train, X_test, y_train, y_test = train_test_split(
                questions, intents, test_size=0.2, random_state=42
            )

            self.intent_classifier.fit(X_train, y_train)

            # Évaluer
            y_pred = self.intent_classifier.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)

            print(f"✅ Classificateur d'intentions entraîné (Accuracy: {accuracy:.3f})")
            return accuracy

        except Exception as e:
            print(f"❌ Erreur entraînement classificateur: {e}")
            return 0

    def predict_intent(self, question):
        """Prédit l'intention d'une question"""
        if self.intent_classifier:
            try:
                return self.intent_classifier.predict([question])[0]
            except:
                return "unknown"
        return "unknown"

# Initialiser le module d'apprentissage
learning_module = LearningModule()

# =============================================================================
# MODULE NLP AVANCÉ - Analyse linguistique approfondie
# =============================================================================

class NLPModule:
    """Module NLP pour analyse linguistique avancée"""

    def __init__(self):
        self.sentiment_analyzer = None
        self.ner_model = None
        self.init_models()

    def init_models(self):
        """Initialise les modèles NLP"""
        try:
            # Analyseur de sentiment multilingue
            self.sentiment_analyzer = pipeline(
                "sentiment-analysis",
                model="nlptown/bert-base-multilingual-uncased-sentiment"
            )
            print("✅ Analyseur de sentiment chargé")
        except Exception as e:
            print(f"⚠️ Sentiment analyzer non disponible: {e}")

        try:
            # NER (Named Entity Recognition)
            self.ner_model = pipeline(
                "ner",
                model="Davlan/distilbert-base-multilingual-cased-ner-hrl"
            )
            print("✅ Modèle NER chargé")
        except Exception as e:
            print(f"⚠️ NER model non disponible: {e}")

    def analyze_sentiment(self, text):
        """Analyse le sentiment du texte"""
        if not self.sentiment_analyzer:
            return {"label": "NEUTRAL", "score": 0.5}

        try:
            result = self.sentiment_analyzer(text[:512])[0]
            return result
        except Exception as e:
            print(f"⚠️ Erreur analyse sentiment: {e}")
            return {"label": "NEUTRAL", "score": 0.5}

    def extract_entities(self, text):
        """Extrait les entités nommées"""
        if not self.ner_model:
            return []

        try:
            entities = self.ner_model(text[:512])
            # Grouper les entités
            grouped = []
            current_entity = None

            for ent in entities:
                if ent['entity'].startswith('B-'):
                    if current_entity:
                        grouped.append(current_entity)
                    current_entity = {
                        'text': ent['word'],
                        'type': ent['entity'][2:],
                        'score': ent['score']
                    }
                elif ent['entity'].startswith('I-') and current_entity:
                    current_entity['text'] += ' ' + ent['word']
                    current_entity['score'] = (current_entity['score'] + ent['score']) / 2

            if current_entity:
                grouped.append(current_entity)

            return grouped
        except Exception as e:
            print(f"⚠️ Erreur extraction entités: {e}")
            return []

    def extract_keywords(self, text, top_n=5):
        """Extrait les mots-clés importants"""
        try:
            from sklearn.feature_extraction.text import TfidfVectorizer

            # Tokeniser les phrases
            sentences = nltk.sent_tokenize(text)
            if len(sentences) < 2:
                return []

            # TF-IDF
            vectorizer = TfidfVectorizer(max_features=top_n, stop_words='english')
            tfidf_matrix = vectorizer.fit_transform(sentences)

            # Obtenir les mots-clés
            feature_names = vectorizer.get_feature_names_out()
            scores = tfidf_matrix.sum(axis=0).A1

            keywords = [(feature_names[i], scores[i]) for i in range(len(feature_names))]
            keywords.sort(key=lambda x: x[1], reverse=True)

            return [k[0] for k in keywords[:top_n]]
        except Exception as e:
            print(f"⚠️ Erreur extraction keywords: {e}")
            return []

    def analyze_text_complexity(self, text):
        """Analyse la complexité du texte"""
        try:
            words = text.split()
            sentences = nltk.sent_tokenize(text)

            avg_word_length = np.mean([len(word) for word in words])
            avg_sentence_length = len(words) / len(sentences) if sentences else 0

            # Score de complexité simple
            complexity_score = (avg_word_length * 0.5 + avg_sentence_length * 0.1) / 2

            if complexity_score < 3:
                level = "Simple"
            elif complexity_score < 5:
                level = "Moyen"
            else:
                level = "Complexe"

            return {
                'level': level,
                'avg_word_length': round(avg_word_length, 2),
                'avg_sentence_length': round(avg_sentence_length, 2),
                'complexity_score': round(complexity_score, 2)
            }
        except Exception as e:
            print(f"⚠️ Erreur analyse complexité: {e}")
            return {'level': 'Unknown', 'complexity_score': 0}

    def full_analysis(self, text):
        """Analyse NLP complète"""
        analysis = {
            'sentiment': self.analyze_sentiment(text),
            'entities': self.extract_entities(text),
            'keywords': self.extract_keywords(text),
            'complexity': self.analyze_text_complexity(text)
        }
        return analysis

# Initialiser le module NLP
nlp_module = NLPModule()

# =============================================================================
# CHARGEMENT DES DONNÉES FAQ
# =============================================================================

def load_faq_data():
    """Charge les données FAQ depuis le fichier CSV fourni"""
    try:
        faq_df = pd.read_csv('tuni_speak_faq_sample.csv')
        faq_df.fillna("", inplace=True)
        faq_df['text'] = faq_df.apply(
            lambda row: f"Question: {row['question']} Réponse: {row['answer']}",
            axis=1
        )
        print(f"✅ Données FAQ chargées: {len(faq_df)} entrées")
        print(f"🌍 Langues disponibles: {faq_df['language'].unique()}")
        return faq_df
    except Exception as e:
        print(f"❌ Erreur lors du chargement des données FAQ: {e}")
        return pd.DataFrame(columns=['id', 'language', 'question', 'answer', 'text'])

faq_df = load_faq_data()

# =============================================================================
# INITIALISATION DES MODÈLES
# =============================================================================

model_name = "deepset/xlm-roberta-large-squad2"
try:
    qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)
    print("✅ Modèle QA chargé avec succès")
except Exception as e:
    print(f"❌ Erreur lors du chargement du modèle QA : {e}")
    qa_pipeline = None

try:
    arabert_prep = ArabertPreprocessor(model_name="aubmindlab/bert-base-arabertv02")
    print("✅ Préprocesseur AraBERT chargé")
except Exception as e:
    print(f"⚠️ Arabert non disponible, utilisant normalisation basique. Erreur : {e}")
    def arabert_prep_fallback(text):
        return text.lower().strip()
    arabert_prep = arabert_prep_fallback

try:
    embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    if not faq_df.empty:
        faq_df['normalized_question'] = faq_df.apply(
            lambda row: normalize_text(row['question'], row['language']),
            axis=1
        )
        faq_df['question_embeddings'] = faq_df['normalized_question'].apply(
            lambda x: embedding_model.encode(x) if x and x != "vide" else embedding_model.encode("")
        )
        print("✅ Embeddings créés pour la recherche sémantique")
    else:
        print("⚠️ Aucune donnée FAQ pour créer les embeddings")
except Exception as e:
    print(f"❌ Erreur lors de la création des embeddings: {e}")
    embedding_model = None

# =============================================================================
# FONCTIONS DE BASE
# =============================================================================

def detect_language(text):
    """Détecte le langage avec meilleure détection du darija"""
    try:
        if not text or not isinstance(text, str):
            return "fr"

        arabic_chars = re.compile(r'[\u0600-\u06FF]')
        darija_patterns = [
            r'\b(ch|sh|9|3|7|khouya|lazma|chnouwa|kifech|n3mel|bach|ntsajel|m3ak|fama|wallah|yasser)\b',
            r'\b(ya3tik|saha|yais|mouch|mch|ken|walah|hata|waktah)\b',
            r'\b(bara|tawa|famma|kima|qadda|mta3|bech|t7eb)\b'
        ]

        if arabic_chars.search(text):
            text_lower = text.lower()
            for pattern in darija_patterns:
                if re.search(pattern, text_lower):
                    return "dar"
            return "ar"

        return langdetect.detect(text) if text and text.strip() else "fr"
    except Exception:
        return "fr"

def normalize_text(text, lang):
    """Nettoie et normalise le texte selon la langue"""
    if not text or not isinstance(text, str):
        return "vide"
    text = text.replace("\n", " ").strip()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'[^\w\s]', '', text)
    if lang in ["fr", "dar"]:
        text = text.lower()
    if lang in ["ar", "dar"] and callable(getattr(arabert_prep, 'preprocess', None)):
        text = arabert_prep.preprocess(text)
    return text if text else "vide"

# =============================================================================
# FONCTIONS DE RECHERCHE ET RÉPONSE AMÉLIORÉES
# =============================================================================

def find_similar_faq(query, threshold=0.6):
    """Recherche les FAQ similaires"""
    if not query or not isinstance(query, str) or faq_df.empty or embedding_model is None:
        return None, None, None, 0.0

    lang = detect_language(query)
    normalized_query = normalize_text(query, lang)

    try:
        query_embedding = embedding_model.encode(normalized_query)
        query_embedding = query_embedding.reshape(1, -1)

        faq_embeddings_list = [np.array(emb) for emb in faq_df['question_embeddings'].tolist()]
        if not faq_embeddings_list:
            return None, None, None, 0.0

        faq_embeddings_matrix = np.vstack(faq_embeddings_list)
        similarities = cosine_similarity(query_embedding, faq_embeddings_matrix).flatten()
        most_similar_index = np.argmax(similarities)
        similarity_score = similarities[most_similar_index]

        if similarity_score >= threshold:
            most_similar_faq = faq_df.iloc[most_similar_index]
            return (most_similar_faq['question'],
                    most_similar_faq['answer'],
                    most_similar_faq['language'],
                    float(similarity_score))
        else:
            return None, None, None, similarity_score

    except Exception as e:
        print(f"❌ Error in find_similar_faq: {e}")
        return None, None, None, 0.0

def configure_gemini_api(api_key):
    """Configure la clé API Gemini"""
    global GEMINI_API_KEY
    GEMINI_API_KEY = api_key.strip()
    if GEMINI_API_KEY:
        try:
            genai.configure(api_key=GEMINI_API_KEY)
            # Tester la connexion
            test_model = genai.GenerativeModel('gemini-flash-latest')
            test_response = test_model.generate_content("Test")
            return f"✅ Clé API configurée avec succès ! Modèle testé: {test_model.model_name}"
        except Exception as e:
            return f"❌ Erreur de configuration: {str(e)}"
    else:
        return "⚠️ Veuillez entrer une clé API valide"

def call_generative_ai(query):
    """IA générative qui répond systématiquement en 3 langues"""
    if not GEMINI_API_KEY:
        return "🇹🇳 الدارجة التونسية: عذرًا، مفيش إجابة حاضرة.\n🇸🇦 العربية الفصحى: عذرًا، لا توجد إجابة متاحة حاليًا.\n🇫🇷 Français: Désolé, aucune réponse disponible pour le moment. (Clé API non configurée)", "IA Générative (Clé API manquante)"

    try:
        genai.configure(api_key=GEMINI_API_KEY)

        system_prompt = """Tu es un assistant trilingue spécialisé.
        Pour CHAQUE question, tu DOIS répondre en 3 langues dans cet ordre :
        1. Darija tunisienne (اللهجة التونسية)
        2. Arabe standard (العربية الفصحى)
        3. Français

        Format de réponse OBLIGATOIRE :
        🇹🇳 الدارجة التونسية: [réponse en darija]
        🇸🇦 العربية الفصحى: [réponse en arabre standard]
        🇫🇷 Français: [réponse en français]

        Sois concis et précis dans tes réponses. Si tu ne connais pas la réponse, dis-le clairement."""

        gemini_model = genai.GenerativeModel(
            model_name='gemini-flash-latest',
            system_instruction=system_prompt
        )

        # Configuration de sécurité
        generation_config = {
            "temperature": 0.7,
            "top_p": 0.95,
            "top_k": 40,
            "max_output_tokens": 1024,
        }

        response = gemini_model.generate_content(
            query,
            generation_config=generation_config,
            safety_settings={
                types.HarmCategory.HARM_CATEGORY_HARASSMENT: types.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                types.HarmCategory.HARM_CATEGORY_HATE_SPEECH: types.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            }
        )

        if response.text:
            return response.text.strip()
        else:
            return "🇹🇳 الدارجة التونسية: مفيش إجابة من السيرڤر.\n🇸🇦 العربية الفصحى: لا توجد إجابة من الخادم.\n🇫🇷 Français: Pas de réponse du serveur.", "IA Générative (Erreur)"

    except Exception as e:
        print(f"❌ Erreur détaillée lors de l'appel à l'IA générative: {e}")
        return f"""🇹🇳 الدارجة التونسية: عذرًا، جاري الصيانة التقنية.
🇸🇦 العربية الفصحى: عذرًا، الخدمة قيد الصيانة التقنية.
🇫🇷 Français: Désolé, le service est en maintenance technique.

Erreur: {str(e)[:100]}""", "IA Générative (Erreur)"

def answer_question(query, include_nlp_analysis=False):
    """Fonction principale pour répondre aux questions avec analyse NLP optionnelle"""
    if not query or not isinstance(query, str):
        return "Veuillez entrer une question valide.", "N/A", None

    print(f"\n🔍 Traitement de la question: {query}")
    lang = detect_language(query)
    print(f"🌍 Langue détectée: {lang}")

    # Analyse NLP si demandée
    nlp_analysis = None
    if include_nlp_analysis:
        nlp_analysis = nlp_module.full_analysis(query)
        print(f"🧠 Analyse NLP effectuée")

    # Vérifier dans les FAQ
    if not faq_df.empty and embedding_model is not None:
        faq_question, faq_answer, faq_lang, faq_similarity_score = find_similar_faq(query, threshold=0.6)

        if faq_question and faq_similarity_score > 0.6:
            print(f"✅ FAQ similaire trouvée (Score: {faq_similarity_score:.3f})")

            # Logger l'interaction
            learning_module.log_interaction(query, faq_answer, "FAQ", lang, faq_similarity_score)

            return faq_answer, f"FAQ (Score: {faq_similarity_score:.3f})", nlp_analysis

        print(f"⚠️ Similarité FAQ insuffisante ({faq_similarity_score:.3f})")

    # Utiliser l'IA générative
    print("🤖 Utilisation de l'IA générative...")
    answer = call_generative_ai(query)

    # Logger l'interaction
    learning_module.log_interaction(query, answer, "IA Générative", lang)

    return answer, "IA Générative", nlp_analysis

# =============================================================================
# FONCTIONS POUR LES DOCUMENTS
# =============================================================================

def extract_text_from_pdf(file_path):
    """Extraction des PDF"""
    text = ""
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                page_text = reader.pages[page_num].extract_text() or ""
                page_text = re.sub(r'\s+', ' ', page_text)
                page_text = re.sub(r'\\n', ' ', page_text)
                text += page_text + " "
    except Exception as e:
        print(f"❌ Erreur extraction PDF {file_path}: {e}")
        return ""
    return text.strip()

def extract_text_from_docx(file_path):
    """Extraction des DOCX"""
    text = []
    try:
        document = Document(file_path)
        for paragraph in document.paragraphs:
            if paragraph.text.strip():
                text.append(paragraph.text)
    except Exception as e:
        print(f"❌ Erreur extraction DOCX {file_path}: {e}")
        return ""
    return '\n'.join(text)

def extract_text(file_path):
    """Dispatch l'extraction selon le type de fichier"""
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension == '.docx':
        return extract_text_from_docx(file_path)
    else:
        print(f"⚠️ Type de fichier non supporté: {file_extension}")
        return ""

# =============================================================================
# INTERFACE GRADIO AMÉLIORÉE
# =============================================================================

def gradio_interface(question, uploaded_file, enable_nlp_analysis, show_learning_report):
    """Interface Gradio avec NLP et apprentissage"""
    response_parts = []

    # Afficher le rapport d'apprentissage si demandé
    if show_learning_report:
        report = learning_module.analyze_patterns()
        response_parts.append(report)
        return "\n\n".join(response_parts)

    # Traitement du document
    if uploaded_file is not None:
        print(f"📄 Traitement du document: {uploaded_file.name}")
        try:
            if isinstance(uploaded_file, str):
                file_path = uploaded_file
                file_name = os.path.basename(file_path)
            else:
                file_path = uploaded_file.name if hasattr(uploaded_file, 'name') else str(uploaded_file)
                file_name = os.path.basename(file_path)

            new_text_content = extract_text(file_path)

            if new_text_content and len(new_text_content.strip()) > 50:
                response_parts.append(f"**✅ Document '{file_name}' traité avec succès.**")
                response_parts.append(f"**📊 Contenu extrait :** {len(new_text_content)} caractères")

                # Analyse NLP du document si activée
                if enable_nlp_analysis:
                    doc_analysis = nlp_module.full_analysis(new_text_content[:1000])
                    response_parts.append("\n**🧠 Analyse NLP du document :**")
                    response_parts.append(f"• Sentiment: {doc_analysis['sentiment']['label']}")
                    response_parts.append(f"• Complexité: {doc_analysis['complexity']['level']}")
                    if doc_analysis['keywords']:
                        response_parts.append(f"• Mots-clés: {', '.join(doc_analysis['keywords'])}")

                print(f"✅ Document intégré: {len(new_text_content)} caractères")
            else:
                response_parts.append(f"**❌ Erreur :** Document vide ou impossible à extraire.")

            os.unlink(file_path)

        except Exception as e:
            response_parts.append(f"**❌ Erreur traitement fichier :** {str(e)}")

    # Traitement de la question
    if question and question.strip():
        print(f"❓ Question reçue: {question}")
        answer, source, nlp_analysis = answer_question(question, enable_nlp_analysis)

        response_parts.append(f"**❓ Votre question :** {question}")
        response_parts.append(f"**💡 Réponse :** {answer}")
        response_parts.append(f"**📚 Source :** {source}")

        # Afficher l'analyse NLP si disponible
        if nlp_analysis and enable_nlp_analysis:
            response_parts.append("\n**🧠 Analyse NLP de votre question :**")
            response_parts.append(f"• Sentiment: {nlp_analysis['sentiment']['label']} (Score: {nlp_analysis['sentiment']['score']:.3f})")
            response_parts.append(f"• Complexité: {nlp_analysis['complexity']['level']}")

            if nlp_analysis['entities']:
                entities_str = ', '.join([f"{e['text']} ({e['type']})" for e in nlp_analysis['entities']])
                response_parts.append(f"• Entités détectées: {entities_str}")

            if nlp_analysis['keywords']:
                response_parts.append(f"• Mots-clés: {', '.join(nlp_analysis['keywords'])}")

        print(f"✅ Réponse générée - Source: {source}")

    if not response_parts:
        response_parts.append("**ℹ️ Veuillez :**\n• Poser une question\n• OU uploader un document\n• OU voir le rapport d'apprentissage !")

    return "\n\n".join(response_parts)

# Créer l'interface Gradio
print("🚀 Création de l'interface Gradio améliorée...")
interface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(
            label="Posez votre question",
            placeholder="Ex: Comment m'inscrire à l'université ?",
            lines=2
        ),
        gr.File(
            label="📤 Uploader un document (PDF/DOCX)",
            file_types=[".pdf", ".docx"]
        ),
        gr.Checkbox(
            label="🧠 Activer l'analyse NLP avancée",
            value=False
        ),
        gr.Checkbox(
            label="📊 Afficher le rapport d'apprentissage",
            value=False
        )
    ],
    outputs=gr.Textbox(
        label="Réponse du système",
        lines=15,
        show_copy_button=True
    ),
    title="🇹🇳 TuniSpeak - Assistant Trilingue avec IA & Apprentissage",
    description=f"""
    **✨ VERSION COMPLÈTE avec NLP et Machine Learning ✨**
    ✓ {len(faq_df)} questions FAQ chargées
    ✓ Analyse NLP avancée (sentiment, entités, mots-clés)
    ✓ Module d'apprentissage continu
    ✓ {len(learning_module.interactions_log)} interactions enregistrées
    ✓ Support trilingue (français, arabe, darija)
    """,
    examples=[
        ["Comment m'inscrire à l'université ?", None, True, False],
        ["كيف أسجل في الجامعة؟", None, False, False],
        ["Quels sont les frais de scolarité ?", None, True, False],
        ["", None, False, True]  # Voir le rapport
    ]
)

# =============================================================================
# APPLICATION FASTAPI AMÉLIORÉE
# =============================================================================

app = FastAPI(title="TuniSpeak API", version="3.0")

os.makedirs("/content/templates", exist_ok=True)

html_content = '''
<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🇹🇳 TuniSpeak - Assistant Trilingue IA</title>
    <style>
        * { box-sizing: border-box; margin: 0; padding: 0; }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 1000px;
            margin: 0 auto;
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
        }
        h1 {
            color: #2c3e50;
            text-align: center;
            margin-bottom: 10px;
        }
        .subtitle {
            text-align: center;
            color: #7f8c8d;
            margin-bottom: 20px;
        }
        .badge {
            display: inline-block;
            background: #3498db;
            color: white;
            padding: 5px 12px;
            border-radius: 15px;
            font-size: 12px;
            margin: 0 5px;
        }
        .stats {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin: 20px 0;
        }
        .stat-card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
        }
        .stat-number {
            font-size: 32px;
            font-weight: bold;
            margin: 10px 0;
        }
        .stat-label {
            font-size: 14px;
            opacity: 0.9;
        }
        .input-group {
            margin-bottom: 20px;
        }
        label {
            display: block;
            margin-bottom: 8px;
            font-weight: 600;
            color: #2c3e50;
        }
        textarea, input[type="file"] {
            width: 100%;
            padding: 12px;
            border: 2px solid #ddd;
            border-radius: 8px;
            font-size: 16px;
            transition: border-color 0.3s;
        }
        textarea:focus, input[type="file"]:focus {
            border-color: #3498db;
            outline: none;
        }
        textarea {
            height: 120px;
            resize: vertical;
        }
        .checkbox-group {
            margin: 15px 0;
        }
        .checkbox-label {
            display: flex;
            align-items: center;
            margin: 10px 0;
            cursor: pointer;
        }
        .checkbox-label input {
            margin-right: 10px;
            width: 20px;
            height: 20px;
            cursor: pointer;
        }
        button {
            background: #3498db;
            color: white;
            border: none;
            padding: 15px 30px;
            font-size: 18px;
            border-radius: 8px;
            cursor: pointer;
            width: 100%;
            transition: background 0.3s;
            margin-top: 10px;
        }
        button:hover {
            background: #2980b9;
        }
        button.secondary {
            background: #95a5a6;
        }
        button.secondary:hover {
            background: #7f8c8d;
        }
        #response {
            margin-top: 30px;
            padding: 20px;
            background: #f8f9fa;
            border-radius: 8px;
            border-left: 4px solid #3498db;
            white-space: pre-wrap;
            max-height: 600px;
            overflow-y: auto;
        }
        #loading {
            display: none;
            text-align: center;
            padding: 20px;
            color: #3498db;
        }
        .features {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin: 20px 0;
        }
        .feature {
            text-align: center;
            padding: 15px;
            background: #f8f9fa;
            border-radius: 8px;
            border: 1px solid #e9ecef;
        }
        .feature-icon {
            font-size: 24px;
            margin-bottom: 10px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🇹🇳 TuniSpeak Assistant IA</h1>
        <div class="subtitle">
            <span class="badge">🧠 NLP</span>
            <span class="badge">📊 ML</span>
            <span class="badge">🌍 Trilingue</span>
        </div>

        <div class="stats">
            <div class="stat-card">
                <div class="stat-label">Questions FAQ</div>
                <div class="stat-number">{{ faq_count }}</div>
            </div>
            <div class="stat-card">
                <div class="stat-label">Interactions</div>
                <div class="stat-number">{{ interactions_count }}</div>
            </div>
            <div class="stat-card">
                <div class="stat-label">Feedbacks</div>
                <div class="stat-number">{{ feedback_count }}</div>
            </div>
        </div>

        <div class="features">
            <div class="feature">
                <div class="feature-icon">🧠</div>
                <div>Analyse NLP</div>
            </div>
            <div class="feature">
                <div class="feature-icon">📚</div>
                <div>Base FAQ</div>
            </div>
            <div class="feature">
                <div class="feature-icon">📄</div>
                <div>Documents</div>
            </div>
            <div class="feature">
                <div class="feature-icon">📊</div>
                <div>Apprentissage</div>
            </div>
        </div>

        <div class="input-group">
            <label for="question">💬 Votre question :</label>
            <textarea id="question" placeholder="Posez votre question en français, arabe ou darija..."></textarea>
        </div>

        <div class="input-group">
            <label for="fileUpload">📤 Document (PDF/DOCX) :</label>
            <input type="file" id="fileUpload" accept=".pdf,.docx">
        </div>

        <div class="checkbox-group">
            <label class="checkbox-label">
                <input type="checkbox" id="enableNLP">
                🧠 Activer l'analyse NLP avancée
            </label>
        </div>

        <button onclick="submitQuestion()">🚀 Envoyer</button>
        <button class="secondary" onclick="showLearningReport()">📊 Rapport d'Apprentissage</button>

        <div id="loading">
            <div>⏳ Traitement en cours...</div>
        </div>

        <div id="response"></div>
    </div>

    <script>
        async function submitQuestion() {
            const question = document.getElementById("question").value;
            const fileInput = document.getElementById("fileUpload");
            const file = fileInput.files[0];
            const enableNLP = document.getElementById("enableNLP").checked;
            const responseDiv = document.getElementById("response");
            const loadingDiv = document.getElementById("loading");

            if (!question.trim() && !file) {
                responseDiv.innerHTML = "<strong>❌ Attention :</strong> Veuillez poser une question ou uploader un document.";
                return;
            }

            loadingDiv.style.display = "block";
            responseDiv.innerHTML = "";

            const formData = new FormData();
            if (question.trim()) formData.append("question", question);
            if (file) formData.append("uploaded_file", file);
            formData.append("enable_nlp", enableNLP);

            try {
                const response = await fetch("/qa", {
                    method: "POST",
                    body: formData
                });

                if (!response.ok) throw new Error(`HTTP ${response.status}`);

                const data = await response.json();
                let htmlResponse = data.answer.replace(/\\n/g, '<br>');
                responseDiv.innerHTML = htmlResponse;

            } catch (error) {
                console.error("Error:", error);
                responseDiv.innerHTML = `<strong>❌ Erreur :</strong> ${error.message}`;
            } finally {
                loadingDiv.style.display = "none";
            }
        }

        async function showLearningReport() {
            const responseDiv = document.getElementById("response");
            const loadingDiv = document.getElementById("loading");

            loadingDiv.style.display = "block";
            responseDiv.innerHTML = "";

            try {
                const response = await fetch("/learning-report");
                if (!response.ok) throw new Error(`HTTP ${response.status}`);

                const data = await response.json();
                responseDiv.innerHTML = data.report.replace(/\\n/g, '<br>');

            } catch (error) {
                console.error("Error:", error);
                responseDiv.innerHTML = `<strong>❌ Erreur :</strong> ${error.message}`;
            } finally {
                loadingDiv.style.display = "none";
            }
        }
    </script>
</body>
</html>
'''

with open("/content/templates/index.html", "w", encoding="utf-8") as f:
    f.write(html_content)

templates = Jinja2Templates(directory="/content/templates")

@app.post("/qa")
async def qa_endpoint(
    question: str = Form(""),
    uploaded_file: UploadFile = File(None),
    enable_nlp: bool = Form(False)
):
    """Endpoint avec support NLP"""
    response_text = ""

    if uploaded_file and uploaded_file.filename:
        try:
            with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(uploaded_file.filename)[1]) as tmp_file:
                content = await uploaded_file.read()
                tmp_file.write(content)
                tmp_path = tmp_file.name

            new_text_content = extract_text(tmp_path)

            if new_text_content and len(new_text_content.strip()) > 50:
                response_text += f"**✅ Document '{uploaded_file.filename}' traité.**\n\n"

                if enable_nlp:
                    doc_analysis = nlp_module.full_analysis(new_text_content[:1000])
                    response_text += f"**🧠 Analyse NLP:** Sentiment: {doc_analysis['sentiment']['label']}, Complexité: {doc_analysis['complexity']['level']}\n\n"

            os.unlink(tmp_path)

        except Exception as e:
            response_text += f"**❌ Erreur fichier:** {str(e)}\n\n"

    if question and question.strip():
        try:
            answer, source, nlp_analysis = answer_question(question, enable_nlp)
            response_text += f"**❓ Question:** {question}\n\n"
            response_text += f"**💡 Réponse:** {answer}\n\n"
            response_text += f"**📚 Source:** {source}\n\n"

            if nlp_analysis and enable_nlp:
                response_text += f"**🧠 Analyse NLP:** Sentiment: {nlp_analysis['sentiment']['label']}, Complexité: {nlp_analysis['complexity']['level']}\n"

        except Exception as e:
            response_text += f"**❌ Erreur:** {str(e)}\n\n"

    if not response_text.strip():
        response_text = "**ℹ️ Veuillez poser une question ou uploader un document.**"

    return {"answer": response_text}

@app.get("/learning-report")
async def learning_report():
    """Endpoint pour le rapport d'apprentissage"""
    report = learning_module.analyze_patterns()
    return {"report": report}

@app.get("/", response_class=HTMLResponse)
async def get_index(request: Request):
    return templates.TemplateResponse("index.html", {
        "request": request,
        "faq_count": len(faq_df),
        "interactions_count": len(learning_module.interactions_log),
        "feedback_count": len(learning_module.feedback_log)
    })

@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "service": "TuniSpeak API v3.0",
        "features": ["NLP", "ML", "FAQ"],
        "faq_count": len(faq_df),
        "interactions": len(learning_module.interactions_log)
    }

# =============================================================================
# LANCEMENT
# =============================================================================

def launch_gradio():
    print("🚀 Lancement de Gradio...")
    return interface.launch(share=True, quiet=True, inbrowser=False)

def launch_fastapi():
    try:
        os.system("fuser -k 8000/tcp 2>/dev/null || true")
        time.sleep(2)

        print("🚀 Lancement FastAPI...")
        config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)

        try:
            ngrok_token = userdata.get("NGROK_AUTH_TOKEN")
            if ngrok_token:
                ngrok.set_auth_token(ngrok_token)
                public_url = ngrok.connect(8000).public_url
                print(f"🌍 Application: {public_url}")
        except:
            print("🌐 Local: http://localhost:8000")

        loop = asyncio.get_event_loop()
        loop.run_until_complete(server.serve())

    except Exception as e:
        print(f"❌ Erreur: {e}")

# Tests
print("\n" + "="*60)
print("🧪 TESTS")
print("="*60)

test_questions = [
    "Comment m'inscrire à l'université ?",
    "كيف أسجل في الجامعة؟",
    "Quels sont les frais de scolarité ?"
]

for question in test_questions:
    print(f"\n--- Test: {question} ---")
    answer, source, nlp_analysis = answer_question(question, True)
    print(f"Réponse: {answer[:100]}...")
    print(f"Source: {source}")
    if nlp_analysis:
        print(f"Sentiment: {nlp_analysis['sentiment']['label']}")

print("\n" + "="*60)
print("✅ SYSTÈME PRÊT avec NLP et ML !")
print("="*60)
print(f"📊 FAQ: {len(faq_df)} | Interactions: {len(learning_module.interactions_log)}")

gradio_thread = threading.Thread(target=launch_gradio, daemon=True)
gradio_thread.start()

import time
time.sleep(3)
launch_fastapi()

🔧 Initialisation du système TuniSpeak avec NLP et ML...


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✅ NLTK resources downloaded


Device set to use cpu


✅ Analyseur de sentiment chargé


...

🚀 Lancement FastAPI...
INFO:     172.31.14.164:0 - "GET /assets/index-CCA5X64I.css HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/index-DputZZxm.js HTTP/1.1" 200 OK


INFO:     Started server process [139]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🌍 Application: https://pseudocartilaginous-secantly-helen.ngrok-free.dev
INFO:     172.31.14.164:0 - "GET /assets/svelte/svelte.js HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/Index-CDBfCghh.js HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/init-CYI6iZYC.js HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/init-DV_66Kck.css HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/MarkdownCode-Ca9FjBJK.css HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/FullscreenButton-BYduS5IX.css HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/StreamingBar-Dq7cCzkm.css HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/Index-BkyjjaNE.css HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/StreamingBar-BtwDBaPl.js HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/FullscreenButton-D7DmzQQ9.js HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /assets/MarkdownCode-DTX-UkhB.js HTTP/1.1" 200 OK
INFO:     172.31.14.164:0 - "GET /asset

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [139]


KeyboardInterrupt: 